In [82]:
import numpy as np
import numpy
import torch.nn as nn

import argparse

import os
import numpy as np
import torch
from PIL import Image

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from engine import train_one_epoch, evaluate
import utils
import transforms as T
import time

import os
import numpy as np
import torch
from PIL import Image
import torch.nn as nn

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchsummary import summary

from engine import train_one_epoch, evaluate
import utils
import transforms as T

import matplotlib.pyplot as plt
import matplotlib

import numpy as np
import pandas as pd
from tqdm import tqdm

matplotlib.use('TkAgg')

In [73]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [74]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    model.backbone.body.conv1 = nn.Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [75]:
def load_all_label(path):
    replay_name = path.split('/')[-2]
    label_path = path.replace(replay_name + "/", '')
    label_len = len(os.listdir(label_path + replay_name +"/"))
    labels = np.array([])
    for i in range(0, label_len):
        labels = np.append(labels, np.load(label_path + replay_name +"/"+ str(i) + ".npy", allow_pickle=True)[1])

    results = labels

    return results

In [76]:
class PennFudanDataset(object):
    def __init__(self, path, transforms, window_size):
        self.root = path
        self.transforms = transforms
        pth = os.listdir(path)
        self.label_sequences = []

        self.dir_paths = []
        for i in pth:
            if os.path.isdir(path + i):
                self.label_sequences += [load_all_label(path + i +'/')]
                self.dir_paths.append(path + i + '/')
                print(f"Loaded {i}")

        self.window_size = window_size

        self.seq_indexs = []
        start = 0
        for i, seq in enumerate(self.label_sequences):
            end = start + len(seq) - (self.window_size - 1)
            self.seq_indexs.append((i, start, end))
            start = end

    def __len__(self):
        return self.seq_indexs[-1][-1]

    def __getitem__(self, idx):
        # load images and masks
        for i, start, end in self.seq_indexs:
            if idx >= start and idx < end:
                real_idx = idx - start

                data = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[0][0]    # (11, 128, 128)
                masks = np.load(self.dir_paths[i] + '/' + str(real_idx) + ".npy", allow_pickle=True)[1][0]   #  (128, 128)
                masks = np.expand_dims(masks, axis=0)
                break

        input_data = self.preprocessing(data)


        num_objs = 1
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return input_data, target


    def preprocessing(self, data):
        #0 ground 1 air 2 building 3 spell 4 ground 5 air 6 building 7 spell 8 resource 9 vision 10 terrain
        temp = np.zeros([self.window_size, 7, data.shape[2],data.shape[2]])

        temp[:,0] = data[0]
        temp[:,1] = data[1]
        temp[:,2] = data[2]
        temp[:,3] = data[4]
        temp[:,4] = data[5]
        temp[:,5] = data[6]
        temp[:,6] = data[9]

        data = temp
        data = data.reshape(self.window_size*data.shape[1],data.shape[2],-1)
        
        return torch.FloatTensor(data)    

In [77]:
import sys
data_path = f"./test_data3/"
# data_path = f"./test_data/"
# data_path = f"./test_data2/"
pth = os.listdir(data_path)
replay_name = int(pth[0])
dataset = PennFudanDataset(data_path, get_transform(train=False), window_size=1)
dataset_test = PennFudanDataset(data_path, get_transform(train=False),window_size=1)

Loaded 6254
Loaded 6254


In [78]:
# 데이터 셋 샘플링 or Plotting
# Start, End, Step = 200, 3000, 500
dataset_len = len(dataset_test)
Start, End, Step = 0, len(dataset_test), 1
test_img_array = []
test_img_one_channel_array = []
test_target_array = []
for i in range(Start,End,Step):
    img_t, target_t = dataset_test[i]
    test_img_array.append(img_t)
    img_one_channel = img_t.sum(axis=0, keepdim=True)
    test_img_one_channel_array.append(img_one_channel)
    test_target_array.append(target_t)

In [79]:
# plot_len = int(len(test_img_array)/2)
# for i in range(plot_len):
#     plt.subplot(2,plot_len,i+1)
#     plt.imshow(test_img_one_channel_array[i][0].mul(255).byte().detach().numpy().T)
#     plt.subplot(2,plot_len,i+1+plot_len)
#     plt.imshow(test_target_array[i]['masks'][0].mul(255).byte().detach().numpy().T)
# plt.show()

In [80]:
# 학습 후 prediction 출력
num_classes = 2
model = get_model_instance_segmentation(num_classes)
model.load_state_dict(torch.load("saved_models/model_10.pth"))
model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
vpx_array = []
vpy_array = []
with torch.no_grad():
    for idx,i in tqdm(enumerate(test_img_array[:2000])):
        prediction = model(torch.unsqueeze(i, 0))
        vpx = int(prediction[0]["boxes"][0][1])*32
        vpy = int(prediction[0]["boxes"][0][0])*32
        vpx_array.append(vpx)
        vpy_array.append(vpy)        

20it [00:48,  2.45s/it]

In [ ]:
dataset_len = 2000
temp = np.zeros((dataset_len,1))
for i in range (0,dataset_len):
    temp[i] = int(i*8)
temp2 = np.zeros((int(temp.max()),1))
for i in range(0, int(temp.max())):
    temp2[i] = i
    
dataset_temp = pd.DataFrame({"frame": temp[:,0],"vpx": vpx_array[:],"vpy": vpy_array[:]})
dataset_temp2 = pd.DataFrame({"frame": temp2[:,0]})
dataset = pd.merge(left = dataset_temp2, right = dataset_temp, how= "left", on = "frame")
dataset = dataset.fillna(method="ffill")
dataset.to_csv("./saved_xy/" + str(replay_name) + ".rep.vpd", header= True, index=False)
dataset.to_csv("C:/TM/starcraft/bwapi-data/read/preset/" + str(replay_name) + ".rep.vpd", header= True, index=False)

print("saved")

In [34]:
temp.max()

28320.0

In [56]:
dataset

,frame,vpx,vpy
0,0.0,0.0,32.0
1,1.0,0.0,32.0
2,2.0,0.0,32.0
3,3.0,0.0,32.0
4,4.0,0.0,32.0
...,...,...,...
28315,28315.0,1408.0,960.0
28316,28316.0,1408.0,960.0
28317,28317.0,1408.0,960.0
28318,28318.0,1408.0,960.0


In [42]:
dataset_temp

,frame,vpx,vpy
0,0.0,0,32
1,8.0,0,32
2,16.0,0,32
3,24.0,0,32
4,32.0,3424,3616
...,...,...,...
3536,28288.0,2144,1184
3537,28296.0,928,1152
3538,28304.0,1376,960
3539,28312.0,1408,960


In [43]:
dataset_temp2

,frame
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
28315,0.0
28316,0.0
28317,0.0
28318,0.0


In [44]:
temp2

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [28]:
for i in range(0, temp.max()):
    temp2[i] = int(i)

array([[0.0000e+00],
       [8.0000e+00],
       [1.6000e+01],
       ...,
       [2.8304e+04],
       [2.8312e+04],
       [2.8320e+04]])

In [20]:
with torch.no_grad():
    for idx,i in enumerate(test_img_array):
        prediction = model(torch.unsqueeze(i, 0))
        vpx = int(prediction[0]["boxes"][0][1])*32
        vpy = int(prediction[0]["boxes"][0][0])*32
        print(vpy)

32
32
32
32
3616
3616
3616
3584
3584


KeyboardInterrupt: 

In [24]:
dataset_len = len(dataset_test)

In [25]:
dataset_len

3541

In [24]:
# 인풋, 정답, prediction에 대한 Plotting
# plot_len = int(len(test_img_array)/2)
# for i in range(plot_len):
#     plt.subplot(3,plot_len,i+1)
#     plt.imshow(test_img_one_channel_array[i][0].mul(255).byte().detach().numpy(),"turbo")
#     plt.subplot(3,plot_len,i+1+ plot_len)
#     plt.imshow(test_target_array[i]['masks'][0].mul(255).byte().detach().numpy().T)
#     plt.subplot(3,plot_len,i+1+ plot_len*2)
#     plt.imshow(prediction[i]['masks'].sum(axis=0).mul(255).byte().detach().numpy().T)
# plt.show()